# 1) Prepare Datafiles
### Input Directory Includes: 
<ul> 
    <li>"t2SOT_rawData.csv"
        <ul>
            <li>Spatial Orientation Task</li>
            <li>retreived from GoogleDrive</li>
            <li>rows = participants</li>
            <li>cols = items on task(1-12)</li>
            <li>cells = un-corrected angular error.</li>
        </ul>
    </li>   
    <li>"t2Model_rawData.csv"
        <ul>
            <li>Virtual Silcton - Model Building</li>
            <li>retreived from SILC website download</li>
            <li>rows = participants; 8 rows per participant</li>
            <li>cols = date of test, model number, building name, x(depV1), y(depV2), error.</li>
        </ul>
    </li>
    <li>"t2Recall_rawData.csv"
        <ul>
            <li>Virtual Silcton - Building Recall</li>
            <li>retreived from Qualtrics</li>
            <li>rows = participants</li>
            <li>cols = duration(seconds), Q1-Q8 building recall</li>
        </ul>
    </li>
    <li>"t2Point_rawData.csv" 
         <ul>
            <li>Virtual Silcton - Pointing</li>
            <li>offline files converted to traditional merged output with python script (included below)</li>
            <li>rows = participants; 56 rows per participant</li>
            <li>cols = pointing from, target builing, correct answer, pointing to, angular error</li>
        </ul>
    </li>  
    <li>"t2MRT_SDTstyle.csv"
        <ul>
            <li>Mental Rotation Task</li>
            <li>retreived from SILC website download</li>
            <li>rows = participants</li>
            <li>col = spatial dimensionality test (SDT)-style score</li>
        </ul>
    </li>   
    <li>"t2Demo.csv"
        <ul>
            <li>Demographics</li>
            <li>retreived from GoogleDrive</li>
            <li>rows = participants</li>
            <li>cols = DOB, sex, age in years, age in months, liscence variable (0= no, 1= yes, 2= permit)</li>
        </ul>
    </li>
    <li>"t1Data.csv" 
         <ul>
            <li>Session 1 Datasheet</li>
            <li>includes all data from participants with data at both session 1 and 2.</li>
         </ul>
    </li> 
</ul>


In [ ]:
inFolder <- "InputFiles/"
outFolder <- "OUT_7.6.19/"

install.packages("psych")
install.packages("GGally")
install.packages("BiDimRegression")
install.packages("plyr")
install.packages("Hmisc")
install.packages("ggplot2")

### Spatial Orientation Task (SOT)

In [ ]:
#import raw datafile
t2SOTdat <- read.csv(paste(inFolder, "t2SOT_rawData.csv", sep=""), header = T, stringsAsFactors = F)
#Rename header coloumns
names(t2SOTdat)[2:13] <- sprintf("SOT_item.%s",seq(1:12))

<p> Corrected angular error = absolute value of (correct angle - participant's response angle)
<br> *note: correct angle for each item (1-12) found in "SOT Kid scores" Google Sheet </p>

In [ ]:
correctAngleList <- as.numeric(c(123, 237, 83, 156, 319, 235, 333, 260, 280, 48, 26, 150)) #list of correct angles

for (item in 1:length(correctAngleList)){
  t2SOTdat[,paste("cSOT_item.", as.character(item), sep="")] <- abs(correctAngleList[item] - t2SOTdat[,1+item])
}
rm(item,correctAngleList) #remove temp files

#Calculate average angular error across all 12 corrected SOT items
t2SOTResults <- subset(t2SOTdat, select = c(1, 14:25))
t2SOTResults$t2.SOT_correct <- rowMeans(t2SOTResults[2:13], na.rm = T)

write.csv(t2SOTResults, paste(outFolder,"t2SOT_out.csv",sep="")) #save output as CSV file

### Model Building - Virtual Silcton

In [ ]:
t2Modeldat <- read.csv(paste(inFolder,"t2Model_rawData.csv",sep=""), header = T, colClasses=c(NA, "NULL", "NULL", NA, NA, NA, "NULL"),
                       stringsAsFactors = F)
names(t2Modeldat)<- c("participant","building","depV1","depV2")

#add coloumns for indepV1 and indepV2
#values were found from "Model Building Data Bidimensional Regression Macro.xlsx"
t2Modeldat$indepV1[t2Modeldat$building == "Batty House"] <- 27
t2Modeldat$indepV1[t2Modeldat$building == "Lynch Station"] <- 12
t2Modeldat$indepV1[t2Modeldat$building == "Harris Hall"] <- 215
t2Modeldat$indepV1[t2Modeldat$building == "Harvey House"] <- 294
t2Modeldat$indepV1[t2Modeldat$building == "Golledge Hall"] <- 563
t2Modeldat$indepV1[t2Modeldat$building == "Snow Church"] <- 716
t2Modeldat$indepV1[t2Modeldat$building == "Sauer Center"] <- 624
t2Modeldat$indepV1[t2Modeldat$building == "Tobler Museum"] <- 493

t2Modeldat$indepV2[t2Modeldat$building == "Batty House"] <- 272
t2Modeldat$indepV2[t2Modeldat$building == "Lynch Station"] <- 124
t2Modeldat$indepV2[t2Modeldat$building == "Harris Hall"] <- 226
t2Modeldat$indepV2[t2Modeldat$building == "Harvey House"] <- 485
t2Modeldat$indepV2[t2Modeldat$building == "Golledge Hall"] <- 291
t2Modeldat$indepV2[t2Modeldat$building == "Snow Church"] <- 205
t2Modeldat$indepV2[t2Modeldat$building == "Sauer Center"] <- 5
t2Modeldat$indepV2[t2Modeldat$building == "Tobler Museum"] <- 122

#Model Building Total
library(BiDimRegression)
observations <- 8            #Number of observation per participant
participants <- length(unique(t2Modeldat$participant)) #Number of participants

t2ModelResults <- data.frame()
#Go through every combination of observations and participants
for (i in 1:(observations*participants))
{
  #If we are on the last value of a new participants
  if (i%%observations==0)
  { 
    temp_results <- BiDimRegression(t2Modeldat[c((i-(observations-1)):i),])
    t2ModelResults[i/observations,1] <- as.character(i/observations)
    t2ModelResults[i/observations,2] <- temp_results$euclidean.r
    t2ModelResults[i/observations,3] <- temp_results$euclidean.rsqr
    t2ModelResults[i/observations,4] <- temp_results$euclidean.angleDEG
  }
}

#Model Building Within-Route 
#create a variable 'routeA' that codes which route each building was on (A=1, B=0)
t2Modeldat$routeA[t2Modeldat$building == "Batty House"] <- 1
t2Modeldat$routeA[t2Modeldat$building == "Lynch Station"] <- 1
t2Modeldat$routeA[t2Modeldat$building == "Harris Hall"] <- 1
t2Modeldat$routeA[t2Modeldat$building == "Harvey House"] <- 1
t2Modeldat$routeA[t2Modeldat$building == "Golledge Hall"] <- 0
t2Modeldat$routeA[t2Modeldat$building == "Snow Church"] <- 0
t2Modeldat$routeA[t2Modeldat$building == "Sauer Center"] <- 0
t2Modeldat$routeA[t2Modeldat$building == "Tobler Museum"] <- 0
#seperate dataframes
t2Modeldat_RouteA <- t2Modeldat[t2Modeldat$routeA == 1,]
t2Modeldat_RouteB <- t2Modeldat[t2Modeldat$routeA == 0,]

observations <- 4            #re-set number of observation per participant
#Route A
for (i in 1:(observations*participants))
{
  #If we are on the last value of a new participants
  if (i%%observations==0)
  { 
    temp_results <- BiDimRegression(t2Modeldat_RouteA[c((i-(observations-1)):i),])
    t2ModelResults[i/observations,5] <- temp_results$euclidean.r
    t2ModelResults[i/observations,6] <- temp_results$euclidean.rsqr
    t2ModelResults[i/observations,7] <- temp_results$euclidean.angleDEG
  }
}

#Route B
for (i in 1:(observations*participants))
{
  #If we are on the last value of a new participants
  if (i%%observations==0)
  { 
    temp_results <- BiDimRegression(t2Modeldat_RouteB[c((i-(observations-1)):i),])
    t2ModelResults[i/observations,8] <- temp_results$euclidean.r
    t2ModelResults[i/observations,9] <- temp_results$euclidean.rsqr
    t2ModelResults[i/observations,10] <- temp_results$euclidean.angleDEG
  }
}

colnames(t2ModelResults) <- c('participant','total.Euclidean_R','total.Euclidean_R2','total.Euclidean_Angle',
                            'routeA.Euclidean_R','routeA.Euclidean_R2','routeA.Euclidean_Angle',
                            'routeB.Euclidean_R','routeB.Euclidean_R2','routeB.Euclidean_Angle')
rm(i,observations,participants,temp_results)#removes temporary files
t2ModelResults[1] <-unique(t2Modeldat$participant)

#Average R2 for routes A & B to get witin-route model building score
t2ModelResults$within.Euclidean_R2 <- rowMeans(t2ModelResults[,c("routeA.Euclidean_R2","routeB.Euclidean_R2")])

write.csv(t2ModelResults, paste(outFolder,"t2Model_out.csv",sep=""), row.names = FALSE) #save output as CSV file

### Pointing Task - Virtual Silcton

<p><strong>*Optional*</strong>: run this code-chunk if pointing data was collected from <strong>offline virtual silcton</strong>. It is a python script which takes offline files and transfers them into prefered output file.
<br><br> If offline files are already converted into merged output, skip this step.</p>

In [ ]:
import os

#these are the answers in the following order:
#Batty, Lynch, Harris, Harvey, Golledge, Snow, Sauer, Tobler by row and column
buildingName = {0:"Batty", 1:"Lynch", 2:"Harris", 3:"Harvey", 4:"Golledge", 5:"Snow", 6:"Sauer", 7:"Tobler"}
correctAns = {	
				"Batty": [0,47.32798389,37.5286373,93.19423766,51.76768154,48.19947553,30.3316951,37.30802198],
				"Lynch": [130.6542463,0,7.021704671,48.19593801,3.323863738,8.968174613,29.20903342,17.95076442],
				"Harris":[83.35216568,114.5627943,0,2.146796904,77.71082192,84.7281122,110.6574336,99.95984558],
				"Harvey":[46.54689912,32.28246429,4.196894849,0,64.9530254,66.19140238,46.29205396,42.14615863],
				"Golledge":[173.5694617,173.9683139,176.2022806,139.9309368,0,1.204134401,76.21088344,147.0284471],
				"Snow":[90.64700942,79.44851915,88.09346154,119.6929579,125.8431145,0,0.279262352,63.69277617,],
				"Sauer":[6.744508622,17.76205492,4.240040564,23.09134403,51.85616181,89.63525726,0,2.713388186,],
				"Tobler":[171.0086651,156.2355625,173.2852643,147.248191,67.30353066,38.78279809,20.69981416,0]
			}	
#corrects an answer for pointing judgments		   
def test(y, z):
	ans = abs(y-z)
	if ans > 180:
		return 360-ans
	else:
		return ans
	
def parseData(fileName):
	with open (fileName, "r") as myfile:
		data = myfile.readlines()
	readPointingSet=[]
	output_file = fileName.rstrip(".txt") + "_output.csv"
	outfile = open(output_file, 'w')
	outfile.write("Filename,Pointing From,Target Building,Correct Ans,Pointing To,Angular Error\n")
	for line in data:
		#startPointingSet() targetBuildingIndicesRemaining: 1,2,3,4,5,6,7
		if "startPointingSet()" in line:
			print "\n=================================================="
			readPointingSet=[]
			splitLine = line.split(' ')
			#converting string of integer to list of integers
			buildingNos = list(eval(splitLine[2]))
			print "targetBuildingIndicesRemaining -",buildingNos
			#finding the missing building index
			standingAt = list(set(buildingName.keys()) - set(buildingNos))
			readPointingSet.append(standingAt[0])
			print "Standing At :",buildingName[standingAt[0]]
			#outfile.write("\n")
			#output.append[buildingName[standingAt[0]]]
		if "OnGUI()" in line:
			splitLine = line.split(' ')
			if len(splitLine) > 2 and len(splitLine[2]) > 2:
				buildingNos = list(eval(splitLine[2].rstrip("\n"))) + readPointingSet
				facingAt = list(set(buildingName.keys()) - set(buildingNos))
				readPointingSet.append(facingAt[0])
				print "Target building #%s - %s:" %(facingAt[0], buildingName[facingAt[0]])
				print "correct Answer is ", correctAns[buildingName[standingAt[0]]][facingAt[0]]
				outfile.write(fileName)
				outfile.write(",")
				outfile.write(buildingName[standingAt[0]])
				outfile.write(",")
				outfile.write(buildingName[facingAt[0]])
				outfile.write(",")
				outfile.write(str(correctAns[buildingName[standingAt[0]]][facingAt[0]]))
				outfile.write(",")
			elif len(splitLine) > 2 and len(splitLine[2]) == 2:
				
				buildingNos = readPointingSet + [splitLine[2].rstrip("\n")]
				facingAt = list(set(buildingName.keys()) - set(buildingNos))
				readPointingSet.append(facingAt[0])
				print "Target building #%s - %s:" %(facingAt[0], buildingName[facingAt[0]])
				print "correct Answer is ", correctAns[buildingName[standingAt[0]]][facingAt[0]]
				outfile.write(fileName)
				outfile.write(",")
				outfile.write(buildingName[standingAt[0]])
				outfile.write(",")
				outfile.write(buildingName[facingAt[0]])
				outfile.write(",")
				outfile.write(str(correctAns[buildingName[standingAt[0]]][facingAt[0]]))
				outfile.write(",")
			else:
				print "targetBuildingIndicesRemaining is None"
				facingAt = list(set(buildingName.keys()) - set(readPointingSet))
				readPointingSet.append(facingAt[0])
				print "Target building #%s - %s:" %(facingAt[0], buildingName[facingAt[0]])
				print "correct Answer is ", correctAns[buildingName[standingAt[0]]][facingAt[0]]
				outfile.write(fileName)
				outfile.write(",")
				outfile.write(buildingName[standingAt[0]])
				outfile.write(",")
				outfile.write(buildingName[facingAt[0]])
				outfile.write(",")
				outfile.write(str(correctAns[buildingName[standingAt[0]]][facingAt[0]]))
				outfile.write(",")
		if "pointing angle:" in line:
			pointingAngle = line.split(' ')
			print "participant pointing angle is: %s" %(pointingAngle[2].rstrip("\n"))
			print "ABS Error",test(float(pointingAngle[2]),correctAns[buildingName[standingAt[0]]][facingAt[0]])
			outfile.write(str(pointingAngle[2].rstrip("\n")))
			outfile.write(",")
			outfile.write(str(test(float(pointingAngle[2].rstrip("\n")),correctAns[buildingName[standingAt[0]]][facingAt[0]])))
			outfile.write("\n")
	outfile.close()

fileList = os.listdir(".")
for fileName in fileList:
	if '.txt' in fileName:
		print "\n################### Parsing File %s ###################\n" %(fileName)
		parseData(fileName)
#parseData()

In [ ]:
#Combine Python output into merged pointing datafile
file_list <- list.files(path= inFolder, full.names = T)
for (file in file_list){
  t2Pointdat <- do.call("rbind.fill",lapply(file_list, FUN=function(files){fread(files, select = c(1:6),fill=TRUE)}))
}
#Give dataset headers
colnames(t2Pointdat) <- c("participant","Pointing.From","Target.Building","Correct.Ans","Pointing.To","Angular.Error")
write.csv(t2Pointdat, paste(inFolder,"t2Point_rawData.csv",sep=""))

<p><strong>If offline files are already converted into merged output, start here:</strong></p>

In [ ]:
t2Pointdat <- read.csv(paste(inFolder, "t2Point_rawData.csv", sep=""), header = T, stringsAsFactors=FALSE)

routeAbuilds <- c("Batty", "Lynch", "Harris", "Harvey")
routeBbuilds <- c("Golledge", "Snow", "Sauer","Tobler")

#Create variable that identifies if point trial was within- or between-route
for (row in 1:nrow(t2Pointdat)){
  #if start & target = same route
  if ((t2Pointdat[row, "Pointing.From"] %in% routeAbuilds && t2Pointdat[row, "Target.Building"] %in% routeAbuilds) |
      (t2Pointdat[row, "Pointing.From"] %in% routeBbuilds && t2Pointdat[row, "Target.Building"] %in% routeBbuilds)){
    t2Pointdat[row, "withinTrial"] <- 1 #within route
  } else {
    t2Pointdat[row, "withinTrial"] <- 0 #between route
  }
}

#Calculate Between-Route Pointing Error
t2Pointdat_Btwn <- t2Pointdat[t2Pointdat$withinTrial == 0,]
t2Pointdat_Wthn <- t2Pointdat[t2Pointdat$withinTrial == 1,]

plist <- unique(t2Pointdat_Btwn$participant)
t2PointResults <- data.frame()

for (p in 1:length(plist)){
  tempdat <- t2Pointdat_Btwn[t2Pointdat_Btwn$participant == plist[p],]
  t2PointResults[p, "participant"] <- plist[p]
  t2PointResults[p, "numTrials_Btwn"] <- nrow(tempdat)
  t2PointResults[p, "angularErr_Btwn"] <- mean(tempdat$Angular.Error)
  
  tempdat2 <- t2Pointdat_Wthn[t2Pointdat_Wthn$participant == plist[p],]
  t2PointResults[p, "numTrials_Wthn"] <- nrow(tempdat2)
  t2PointResults[p, "angularErr_Wthn"] <- mean(tempdat2$Angular.Error)
}

rm(p,tempdat,tempdat2, row, plist)#removes temporary files

write.csv(t2Pointdat, paste(outFolder,"t2Point_out.csv",sep="")) #save output as CSV file

### Building Recall - Virtual Silcton

In [ ]:
t2Recalldat <- read.csv(paste(inFolder, "t2Recall_rawData.csv", sep=""), header = T, stringsAsFactors = F)

recallAnswers <- c("Harvey House", "Golledge Hall", "Snow Church", "Lynch Station",
                   "Sauer Center", "Harris Hall", "Tobler Museum", "Batty House")

t2RecallResults <- data.frame()
for (p in 1:nrow(t2Recalldat)){
  for (i in 1:length(recallAnswers)){
    t2RecallResults[p, paste("recall.Q",i,sep="")] <- as.numeric(grepl(recallAnswers[i],t2Recalldat[p,2+i],ignore.case=TRUE))
  }
}

t2RecallResults$recallScore <- rowSums(t2RecallResults)
t2RecallResults <- cbind(t2Recalldat$participant, t2RecallResults, stringsAsFactors = F)
names(t2RecallResults)[1] <- "participant"

### Compile all results into one dataframe


In [ ]:
t2MRT <- read.csv(paste(inFolder, "t2MRT_SDTstyle.csv", sep=""), header = T, stringsAsFactors = F)
t2Demo <- read.csv(paste(inFolder, "t2Demo.csv", sep=""), header = T, stringsAsFactors = F)

resultsList <- list(t2Demo, t2RecallResults, t2ModelResults, t2PointResults, t2MRT, t2SOTResults)

#Merge all DFs into 1
for (task in 1:length(resultsList)){
  if (task == 1){
    t2AllResults <- merge(resultsList[task], resultsList[task+1], by="participant", all = T)
  }
  if (task == 2){
    
  }else
    t2AllResults <- merge(t2AllResults, resultsList[task], by="participant", all = T)
}

#Import t1 data
t1AllDat <- read.csv(paste(inFolder, "t1Data.csv", sep=""), header = T, stringsAsFactors = F)
allResults <- merge(t2AllResults, t1AllDat, by="participant", all = T)

allData <- allResults[,c(1:4,6,15,22,30,32,34,35,48,51,54:57,59:60,63:68)]
names(allData) <- c("ID", "t2.Age_years", "DOB", "t2.Age_months","t2.License",
                       "t2.BuildingRecall", "t2.ModelRsq_Total", "t2.ModelRsq_Within",
                       "t2.PointError_Btwn", "t2.PointError_Wthn", "t2.MRT", "t2.SOT",
                       "t1.SOT", "t1.ModelRsq_Within", "t1.BuildingRecall","t1.MRT",
                       "t1.ModelRsq_Total", "t1.PointError_Wthn", "t1.PointError_Btwn",
                       "t1.Age_years", "Female", "t1.SODA_Average", "t1.cluster_both",
                       "t1.cluster_manual", "t1.kmeans_3clusters")

write.csv(allData, paste(outFolder,"LongitudinalSilcton_ReturnData.csv",sep="")) #save output as CSV file

# 2) Summary Statistics

In [ ]:
#load necessary libraries & create dataframes
library('psych')
library('GGally')

sumStats <- subset(allData, select = -c(DOB, ID, Female, t2.License, t1.Age_years, t2.Age_months, t1.SODA_Average, 
                                        t1.cluster_manual, t1.cluster_both, t1.kmeans_3clusters))

t1.t2.Model <- subset(allData, select = c(t1.BuildingRecall, t1.ModelRsq_Total, t1.ModelRsq_Within, 
                                           t2.BuildingRecall, t2.ModelRsq_Total, t2.ModelRsq_Within))
t1.t2.Point <- subset(allData, select = c(t1.PointError_Btwn, t1.PointError_Wthn, t2.PointError_Btwn,
                                           t2.PointError_Wthn))
t2.Point <- subset(allResults, select = c(numTrials_Btwn, numTrials_Wthn))
  
t1.t2.PT_MRT <- subset(allData, select = c(t1.MRT, t1.SOT, t2.MRT, t2.SOT))

### Scatterplots, Correlation Matrices, and Descriptives

In [ ]:
#scatterplots & correlation matrices 
ggpairs(t1.t2.Model, lower = list(continuous=wrap("smooth", method="lm",color="red"), combo="facetdensity"))
ggpairs(t1.t2.Point, lower = list(continuous=wrap("smooth", method="lm",color="red"), combo="facetdensity"))
ggpairs(t1.t2.PT_MRT, lower = list(continuous=wrap("smooth", method="lm",color="red"), combo="facetdensity"))

#Descriptives
describe(t1.t2.Model)
describe(t1.t2.Point)
describe(t1.t2.PT_MRT)

#double check the number of between- and within-route pointing trials
#should be 56 total(n=32 between & n=24 within).
describe(t2.Point)

### Outlier Detection

In [ ]:
#Outlier Detection for T2
boxplot(allData$t2.BuildingRecall, main="T2: Building Recognition", 
        sub=paste("Outlier rows: ", boxplot.stats(allData$t2.BuildingRecall)$out))
boxplot(allData$t2.ModelRsq_Total, main="T2: Model Building Rsq (Total)", 
        sub=paste("Outlier rows: ", boxplot.stats(allData$t2.ModelRsq_Total)$out))
boxplot(allData$t2.ModelRsq_Within, main="T2: Model Building Rsq (Within-Route)", 
        sub=paste("Outlier rows: ", boxplot.stats(allData$t2.ModelRsq_Within)$out))
boxplot(allData$t2.PointError_Btwn, main="T2: Pointing Error (Between-Route)", 
        sub=paste("Outlier rows: ", boxplot.stats(allData$t2.PointError_Btwn)$out))
boxplot(allData$t2.PointError_Wthn, main="T2: Pointing Error (Within-Route)", 
        sub=paste("Outlier rows: ", boxplot.stats(allData$t2.PointError_Wthn)$out))
boxplot(allData$t2.MRT, main="T2: Mental Rotation Task (SDT-style)", 
        sub=paste("Outlier rows: ", boxplot.stats(allData$t2.MRT)$out))
boxplot(allData$t2.SOT, main="T2: Spatial Orientation Task\n (angular error)", 
        sub=paste("Outlier rows: ", boxplot.stats(allData$t2.SOT)$out))

# 3) Tables

### Table 1. Participants by T2 Age & Gender

In [ ]:
library(plyr)
sumStats$t2.Age_years_floor <- as.factor(floor(sumStats$t2.Age_years))

table1 <- ddply(sumStats, c("Female", "t2.Age_years_floor"), summarise,
                N    = length(t2.Age_years_floor))

write.csv(table1, paste(outFolder,"Table1.csv",sep=""))


### Table 2. Descriptive statistics for psychometric and navigation tasks

In [ ]:
navpsych_dat <- subset(sumStats, select = -c(t2.Age_years, Female, t2.Age_years_floor))

library(stringr)
navpsych_dat <- navpsych_dat[,str_sort(names(navpsych_dat), numeric = TRUE)] #organizes variables

#pairwise deletion of outlier for SOT
navpsych_dat[navpsych_dat$t2.SOT == boxplot.stats(allData$t2.SOT)$out, "t2.SOT"] <- NA

table2 <- describe(navpsych_dat)

write.csv(table2, paste(outFolder,"Table2.csv",sep=""))

### Table 3. Correlations among age, performances on VE, and psychometric measures

In [ ]:
table3_dat <- subset(sumStats, select = c(t2.Age_years, t2.BuildingRecall, t2.ModelRsq_Total,
                                          t2.ModelRsq_Within, t2.PointError_Btwn, t2.PointError_Wthn,
                                          t2.MRT, t2.SOT))

#Correlation table with p values
library(Hmisc)
table3 <- rcorr(as.matrix(table3_dat))

write.csv(table3$r, paste(outFolder,"Table3_corr.csv",sep=""))
write.csv(table3$P, paste(outFolder,"Table3_sigs.csv",sep=""))

# 4) Figures

### Figure 1. Cross-sectional trend lines for between-route and within-route pointing trials. 

In [ ]:
fig1 <- ggplot(fig1_dat_sum, aes(x=t2.Age_years_floor, y=PointError, group = trialType))+ 
  geom_errorbar(aes(ymin=PointError-sd, ymax=PointError+sd), width=.1, 
                position=position_dodge(0.05)) +
  geom_line(aes(linetype=trialType)) +
  geom_point(aes(shape=trialType), show.legend=FALSE)+
  labs(title="Plot of Pointing Error by Age", x= "Participant Age (years)",  y = "Error (°)", linetype= "Trial Type")+
  theme_classic() + 
  scale_color_manual(values=c('#999999','#E69F00')) +
  scale_linetype_manual(name = "Trial Type", values = c('solid', 'dashed')) +
  scale_x_discrete(name = "Participant Age (years)", 
                     breaks= c(11:19), 
                     labels= c(sprintf("%s yrs",c(11:19))))+
  scale_y_continuous(name = "Error (°)", 
                     breaks= c(seq(from=0, to=70, by=10)))+
  theme(plot.title = element_text(hjust = 0.5))+
  theme(text=element_text(family="serif"))

fig1


### Figure 2 . Cross-sectional trend lines for within-route and total model building accuracy. 

In [ ]:
fig2_dat1 <- subset(sumStats, select = c(t2.Age_years, t2.ModelRsq_Total))
fig2_dat1$modType <- "Total"
names(fig2_dat1)[2] <- "Accuracy"
fig2_dat2 <- subset(sumStats, select = c(t2.Age_years, t2.ModelRsq_Within))
fig2_dat2$modType <- "Within"
names(fig2_dat2)[2] <- "Accuracy"
fig2_dat <- rbind(fig2_dat1,fig2_dat2)
fig2_dat$t2.Age_years_floor <- as.factor(floor(fig2_dat$t2.Age_years))

#Call summary function
fig2_dat_sum <- data_summary(fig2_dat, varname="Accuracy", 
                             groupnames=c("modType", "t2.Age_years_floor"))

# Line Plot with Error Bars
fig2 <- ggplot(fig2_dat_sum, aes(x=t2.Age_years_floor, y=Accuracy, group = modType))+ 
  geom_errorbar(aes(ymin=Accuracy-sd, ymax=Accuracy+sd), width=.1, 
                position=position_dodge(0.05)) +
  geom_line(aes(linetype=modType)) +
  geom_point(aes(shape=modType), show.legend=FALSE)+
  labs(title="Plot of Model Building Accuracy by Age", x= "Participant Age (years)",  y = "Accuracy (R²)", linetype= "Model Type")+
  theme_classic() + 
  scale_color_manual(values=c('#999999','#E69F00')) +
  scale_linetype_manual(name = "Model Type", values = c('solid', 'dashed')) +
  scale_x_discrete(name = "Participant Age (years)", 
                   breaks= c(11:19), 
                   labels= c(sprintf("%s yrs",c(11:19))))+
  scale_y_continuous(name = "Accuracy (R²)", 
                     breaks= c(seq(from=0, to=90, by=10)))+
  theme(plot.title = element_text(hjust = 0.5))+
  theme(text=element_text(family="serif"))

fig2

### Figure 3 . Vector Plot
<p>Vector plot of the differences in rates of change for between-route and within-route pointing error where vector initial point = Time 1, vector end point = Time 2.</p>

In [ ]:
fig3dat <- subset(allData, select = c(ID, t1.PointError_Btwn, t1.PointError_Wthn,
                                      t2.PointError_Btwn, t2.PointError_Wthn, t1.Age_years, t2.Age_years))

fig3dat$t2AgeGroup[fig3dat$t1.Age_years <= 13] <-  "Age <= 13 years"
fig3dat$t2AgeGroup[fig3dat$t1.Age_years > 13] <-  "Age > 13 years"

overallLT13 <- fig3dat[fig3dat$t2AgeGroup == "Age <= 13 years",]
overallGT13 <- fig3dat[fig3dat$t2AgeGroup == "Age > 13 years",]

#Overall Less Than 13
LT13_x1 <- mean(overallLT13$t1.PointError_Btwn, na.rm = T)
LT13_x2 <- mean(overallLT13$t2.PointError_Btwn, na.rm = T)
LT13_y1 <- mean(overallLT13$t1.PointError_Wthn, na.rm = T)
LT13_y2 <- mean(overallLT13$t2.PointError_Wthn, na.rm = T)
         
#Overall Greater Than 13
GT13_x1 <- mean(overallGT13$t1.PointError_Btwn, na.rm = T)
GT13_x2 <- mean(overallGT13$t2.PointError_Btwn, na.rm = T)
GT13_y1 <- mean(overallGT13$t1.PointError_Wthn, na.rm = T)
GT13_y2 <- mean(overallGT13$t2.PointError_Wthn, na.rm = T)

# Basic scatter plot
fig3 <- ggplot(fig3dat, aes(x=t1.PointError_Btwn, y=t1.PointError_Wthn)) + 
  geom_point(aes(color= t2AgeGroup))+
  labs(title="Differences in Rates of Improvement of Between- and Within-route Pointing Error\n from Time-point 1 to Time-point 2", 
       x= "Between-route pointing error",  y = "Within-route pointing error", color="Age Group")+
  theme_classic()+
  scale_x_continuous(name = "Between-route pointing error", limits=c(0, 75),
                     breaks= c(seq(from=0, to=80, by=10)))+
  scale_y_continuous(name = "Within-route pointing error",limits=c(0, 50), 
                     breaks= c(seq(from=0, to=50, by=10)))+
  geom_segment(aes(x=t1.PointError_Btwn, xend=t2.PointError_Btwn, 
                   y=t1.PointError_Wthn, yend=t2.PointError_Wthn,
                   colour= t2AgeGroup),
               arrow=arrow(length=unit(0.1,"cm"), ends="last", type = "closed"))+
  scale_color_manual(values=c("Age <= 13 years"="grey", "Age > 13 years"="light blue"))+
  
  geom_segment(aes(x=GT13_x1, xend=GT13_x2, y=GT13_y1, yend=GT13_y2), 
               arrow = arrow(length = unit(0.3, "cm"),ends="last", type = "open", angle=30),
               colour="#2360CF", size=1.2)+
  geom_segment(aes(x=LT13_x1, xend=LT13_x2, y=LT13_y1, yend=LT13_y2), 
               arrow = arrow(length = unit(0.3, "cm"),ends="last", type = "open",angle=30),
               colour="black", size=1.2)+
  theme(plot.title = element_text(hjust = 0.5))+
  theme(text=element_text(family="serif"))
fig3

# 5) Regression Models

In [ ]:
#outlier for SOT
sumStats[sumStats$t2.SOT == boxplot.stats(allData$t2.SOT)$out, "t2.SOT"] <- NA

#Z-score variables
sumStats$t2.zSOT <- (sumStats$t2.SOT - mean(sumStats$t2.SOT, na.rm=T))/sd(sumStats$t2.SOT, na.rm=T)
sumStats$t2.zMRT <- (sumStats$t2.MRT - mean(sumStats$t2.MRT, na.rm=T))/sd(sumStats$t2.MRT, na.rm=T)
sumStats$t2.zModelRsq_Total <- (sumStats$t2.ModelRsq_Total - mean(sumStats$t2.ModelRsq_Total, na.rm=T))/sd(sumStats$t2.ModelRsq_Total, na.rm=T)
sumStats$t2.zPointError_Btwn <- (sumStats$t2.PointError_Btwn - mean(sumStats$t2.PointError_Btwn, na.rm=T))/sd(sumStats$t2.PointError_Btwn, na.rm=T)
sumStats$t2.zPointError_Wthn <- (sumStats$t2.PointError_Wthn - mean(sumStats$t2.PointError_Wthn, na.rm=T))/sd(sumStats$t2.PointError_Wthn, na.rm=T)
sumStats$Female.fact <- as.factor(sumStats$Female)

regData <- subset(sumStats, select = c(t2.Age_years, Female.fact, t2.zSOT, t2.zMRT,
                                       t2.zModelRsq_Total, t2.zPointError_Wthn,
                                       t2.zPointError_Btwn))
which(is.na(regData), arr.ind=TRUE)#check for missing dat
regData <- na.omit(regData) #listwise deletion

library(stats)
#Predict Model Building Total at T2
modBuild <- lm(t2.zModelRsq_Total ~ Female.fact + t2.zSOT + t2.zMRT, data=regData)
summary(modBuild)
modBuild.age <- lm(t2.zModelRsq_Total ~ Female.fact + t2.zSOT + t2.zMRT + t2.Age_years, data=regData)
summary(modBuild.age)

withinRoute <- lm(t2.zPointError_Wthn ~ Female.fact + t2.zSOT + t2.zMRT, data=regData)
summary(withinRoute)
withinRoute.age <- lm(t2.zPointError_Wthn ~ Female.fact + t2.zSOT + t2.zMRT + t2.Age_years, data=regData)
summary(withinRoute.age)

btwnRoute <- lm(t2.zPointError_Btwn ~ Female.fact + t2.zSOT + t2.zMRT, data=regData)
summary(btwnRoute)
btwnRoute.age <- lm(t2.zPointError_Btwn ~ Female.fact + t2.zSOT + t2.zMRT + t2.Age_years, data=regData)
summary(btwnRoute.age)

#Combine all regression results into 1 DF and write output
regOut <- rbind(summary(modBuild)$coeff, summary(modBuild.age)$coeff,
                summary(withinRoute)$coeff, summary(withinRoute.age)$coeff,
                summary(btwnRoute)$coeff, summary(btwnRoute.age)$coeff)

write.csv(regOut, paste(outFolder,"t2_Regressions_OUT.csv",sep=""))

#Change in F and Rsq
anova(modBuild, modBuild.age)
anova(withinRoute, withinRoute.age)
anova(btwnRoute, btwnRoute.age)
